In [18]:
from mxnet import nd, gpu, gluon, autograd
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
import time

In [19]:
x = nd.ones((3,4), ctx=gpu())
x


[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
<NDArray 3x4 @gpu(0)>

In [3]:
import numpy
print(numpy.__version__)

1.14.5


In [4]:
import pandas 
print(pandas.__version__)

0.25.3


In [5]:
import mxnet
mxnet.__version__

'1.4.1'

In [6]:
!pip install gluonts

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import gluonts
print(gluonts.__version__)

0.4.0


In [8]:
import pprint
from functools import partial

import pandas as pd

from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer import Trainer

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [12]:
datasets = [
#     "m4_hourly",
#     "m4_daily",
#     "m4_weekly",
#     "m4_monthly",
    "m4_quarterly",
#     "m4_yearly",
]

epochs = 100
num_batches_per_epoch = 100

estimators = [
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch,
            ctx="gpu"
        ),
    ),
]


def evaluate(dataset_name, estimator):
    dataset = get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True,
        cardinality=[
            feat_static_cat.cardinality
            for feat_static_cat in dataset.metadata.feat_static_cat
        ],
    )

    print(f"evaluating {estimator} on {dataset}")

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint.pprint(agg_metrics)

    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict


if __name__ == "__main__":

    results = []
    for dataset_name in datasets:
        for estimator in estimators:
            # catch exceptions that are happening during training to avoid failing the whole evaluation
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))

    df = pd.DataFrame(results)

    sub_df = df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]

    print(sub_df.to_string())


INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_quarterly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepARTrainingNetwork: 1230523


evaluating gluonts.model.deepar._estimator.DeepAREstimator(cardinality=[24000], cell_type="lstm", context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), dropout_rate=0.1, embedding_dimension=None, freq="3M", lags_seq=None, num_cells=40, num_layers=2, num_parallel_samples=100, prediction_length=8, scaling=True, time_features=None, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=mxnet.context.Context("gpu", 0), epochs=100, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=100, patience=10, weight_decay=1e-08), use_feat_dynamic_real=False, use_feat_static_cat=True, use_feat_static_real=False) on TrainDatasets(metadata=<MetaData freq='3M' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='24000'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=8>, train=<gluonts.dataset.common.File

100%|██████████| 100/100 [00:03<00:00, 25.33it/s, avg_epoch_loss=7.97]
INFO:root:Epoch[0] Elapsed time 3.951 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.968326
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.81it/s, avg_epoch_loss=7.47]
INFO:root:Epoch[1] Elapsed time 3.731 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.466114
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.42it/s, avg_epoch_loss=7.23]
INFO:root:Epoch[2] Elapsed time 3.789 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7.230326
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 26.95it/s, avg_epoch_loss=7.19]
INFO:root:Epoch[3] Elapsed time 3.712 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=7.191239
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 27.02it/s, avg_epoch_loss=7.09]
INFO:root:Epoch[4] Elapsed time 3.702 seconds
INFO:root:E

make_evaluation_predictions() got an unexpected keyword argument 'num_eval_samples'


KeyError: "None of [Index(['dataset', 'estimator', 'RMSE', 'mean_wQuantileLoss', 'MASE', 'sMAPE',\n       'MSIS'],\n      dtype='object')] are in the [columns]"